* objective: recommend new products(products were not purchased in previous month) in June 2016
* train model for June-2015. new products for each customer in June-2015 as response variables. Features are characteristics of customers and lag data(which are whether the customers bought 24 products in the previous 5 months and other features) 
* used xgboost multiclass model
* tunned parameters: max_depth, 'min_child_weight', 'subsample, 'colsample_bytree', 'gamma', 'eta'
* obtained probability of each product for customers
* removed products that customers already have bought in May 2016. 
* recommended only popular products. 
* extracted 7 products with highest probabilty for each customers. 

In [1]:
import pandas as pd

import numpy as np

import seaborn as sns

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm

import datetime
import calendar


from IPython.display import Image

#from mlencoders.weight_of_evidence_encoder import WeightOfEvidenceEncoder

import time 
from sklearn.feature_selection import chi2, SelectKBest

from sklearn.model_selection import train_test_split

#from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB

import pickle

#from skmultilearn.problem_transform import BinaryRelevance
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
#from mlxtend.evaluate import confusion_matrix
#from mlencoders.weight_of_evidence_encoder import WeightOfEvidenceEncoder

from sklearn import preprocessing
import xgboost as xgb


from sklearn.metrics import f1_score
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [4]:
root ='/Users/mengruiyin/Dropbox/project2'

In [5]:
new_each_month = {}

In [ ]:
data = 

In [12]:
new_each_month[5] = pd.read_csv(root + '/lag_June15.csv')

FileNotFoundError: File b'/Volumes/My Passport for Mac/lag_June15.csv' does not exist

In [16]:
new_each_month[6] = pd.read_csv('/Volumes/My Passport for Mac/project2/lag_May16_0612.csv')

In [5]:
new_each_month[5].shape

(36703, 227)

In [20]:
new_each_month[6].iloc[0]['ncodpers']

15889

In [25]:
countbyn = pd.DataFrame(new_each_month[6].groupby('ncodpers').size())
countbyn[countbyn.values != 1]

,0
ncodpers,


In [21]:
new_each_month[6][new_each_month[6].ncodpers == 15889]

,ind_ahor_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,...,indext_S,conyuemp_N,conyuemp_S,conyuemp_unknown,indfall_N,indfall_S,segmento_01 - TOP,segmento_02 - PARTICULARES,segmento_03 - UNIVERSITARIO,card number
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,1,0,1,0,0,4.0


In [28]:
June_15_lable = new_each_month[6][product]
June_15_lable['ncodpers'] = new_each_month[6]['ncodpers']
June_15_lable.set_index('ncodpers', inplace = True)
stacked_lables = June_15_lable.stack()
stacked_lables = stacked_lables.reset_index()
stacked_lables.columns = ['ncodpers', 'product', 'new']
stacked_lables

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,ncodpers,product,new
0,15889,ind_ahor_fin_ult1,0.0
1,15889,ind_cco_fin_ult1,0.0
2,15889,ind_cder_fin_ult1,0.0
3,15889,ind_cno_fin_ult1,0.0
4,15889,ind_ctju_fin_ult1,0.0
5,15889,ind_ctma_fin_ult1,0.0
6,15889,ind_ctop_fin_ult1,0.0
7,15889,ind_ctpp_fin_ult1,0.0
8,15889,ind_dela_fin_ult1,0.0
9,15889,ind_ecue_fin_ult1,0.0


In [29]:
target_transform = stacked_lables[stacked_lables.new == 1]

In [34]:
multi_class_June_15 = pd.merge(new_each_month[6], target_transform, on = 'ncodpers', how = 'inner')
multi_class_June_15.index = multi_class_June_15.ncodpers
multi_class_June_15.sort_index()

,ind_ahor_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,...,conyuemp_S,conyuemp_unknown,indfall_N,indfall_S,segmento_01 - TOP,segmento_02 - PARTICULARES,segmento_03 - UNIVERSITARIO,card number,product,new
ncodpers,,,,,,,,,,,,,,,,,,,,,
15889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,1,0,0,4.0,ind_tjcr_fin_ult1,1.0
15929,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,1,0,0,6.0,ind_recibo_ult1,1.0
15952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,1,0,0,3.0,ind_recibo_ult1,1.0
15988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,1,0,0,2.0,ind_tjcr_fin_ult1,1.0
15993,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,1,0,0,6.0,ind_tjcr_fin_ult1,1.0
16008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,1,0,0,1,0,7.0,ind_tjcr_fin_ult1,1.0
16056,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,1,0,1,0,0,5.0,ind_nom_pens_ult1,1.0
16056,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,1,0,1,0,0,5.0,ind_nomina_ult1,1.0
16125,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0,1,1,0,1,0,0,4.0,ind_ctop_fin_ult1,1.0


In [35]:
June_15_X = multi_class_June_15.drop(columns = ['product', 'new'])
June_15_y = multi_class_June_15['product']

In [36]:
June_15_y

ncodpers
15889      ind_tjcr_fin_ult1
15929        ind_recibo_ult1
15952        ind_recibo_ult1
15988      ind_tjcr_fin_ult1
15993      ind_tjcr_fin_ult1
16008      ind_tjcr_fin_ult1
16056      ind_nom_pens_ult1
16056        ind_nomina_ult1
16125      ind_ctop_fin_ult1
16133      ind_tjcr_fin_ult1
16201      ind_nom_pens_ult1
16201        ind_nomina_ult1
16201      ind_tjcr_fin_ult1
16202      ind_tjcr_fin_ult1
16242       ind_cno_fin_ult1
16294      ind_ecue_fin_ult1
16506      ind_nom_pens_ult1
16506        ind_nomina_ult1
16525      ind_ctpp_fin_ult1
16536       ind_cno_fin_ult1
16536      ind_tjcr_fin_ult1
16576       ind_cno_fin_ult1
16680       ind_cno_fin_ult1
16705      ind_tjcr_fin_ult1
16731        ind_recibo_ult1
16787      ind_nom_pens_ult1
16787        ind_nomina_ult1
16826        ind_recibo_ult1
16857       ind_cco_fin_ult1
16988      ind_tjcr_fin_ult1
                 ...        
1553119     ind_cco_fin_ult1
1553120     ind_cco_fin_ult1
1553147     ind_cco_fin_ult1
15531

In [6]:
a = list(new_each_month[5].columns)
a

['ind_cco_fin_ult1',
 'ind_cder_fin_ult1',
 'ind_cno_fin_ult1',
 'ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1',
 'ind_ctop_fin_ult1',
 'ind_ctpp_fin_ult1',
 'ind_deco_fin_ult1',
 'ind_dela_fin_ult1',
 'ind_deme_fin_ult1',
 'ind_ecue_fin_ult1',
 'ind_fond_fin_ult1',
 'ind_hip_fin_ult1',
 'ind_nom_pens_ult1',
 'ind_nomina_ult1',
 'ind_plan_fin_ult1',
 'ind_pres_fin_ult1',
 'ind_reca_fin_ult1',
 'ind_recibo_ult1',
 'ind_tjcr_fin_ult1',
 'ind_valo_fin_ult1',
 'ind_viv_fin_ult1',
 'ncodpers',
 'ind_ahor_fin_ult1_lag1',
 'ind_aval_fin_ult1_lag1',
 'ind_cco_fin_ult1_lag1',
 'ind_cder_fin_ult1_lag1',
 'ind_cno_fin_ult1_lag1',
 'ind_ctju_fin_ult1_lag1',
 'ind_ctma_fin_ult1_lag1',
 'ind_ctop_fin_ult1_lag1',
 'ind_ctpp_fin_ult1_lag1',
 'ind_deco_fin_ult1_lag1',
 'ind_deme_fin_ult1_lag1',
 'ind_dela_fin_ult1_lag1',
 'ind_ecue_fin_ult1_lag1',
 'ind_fond_fin_ult1_lag1',
 'ind_hip_fin_ult1_lag1',
 'ind_plan_fin_ult1_lag1',
 'ind_pres_fin_ult1_lag1',
 'ind_reca_fin_ult1_lag1',
 'ind_tjcr_fin_ult1_lag1',
 

In [10]:
list(new_each_month[5].columns)

['ind_cco_fin_ult1',
 'ind_cder_fin_ult1',
 'ind_cno_fin_ult1',
 'ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1',
 'ind_ctop_fin_ult1',
 'ind_ctpp_fin_ult1',
 'ind_deco_fin_ult1',
 'ind_dela_fin_ult1',
 'ind_deme_fin_ult1',
 'ind_ecue_fin_ult1',
 'ind_fond_fin_ult1',
 'ind_hip_fin_ult1',
 'ind_nom_pens_ult1',
 'ind_nomina_ult1',
 'ind_plan_fin_ult1',
 'ind_pres_fin_ult1',
 'ind_reca_fin_ult1',
 'ind_recibo_ult1',
 'ind_tjcr_fin_ult1',
 'ind_valo_fin_ult1',
 'ind_viv_fin_ult1',
 'ncodpers',
 'ind_ahor_fin_ult1_lag1',
 'ind_aval_fin_ult1_lag1',
 'ind_cco_fin_ult1_lag1',
 'ind_cder_fin_ult1_lag1',
 'ind_cno_fin_ult1_lag1',
 'ind_ctju_fin_ult1_lag1',
 'ind_ctma_fin_ult1_lag1',
 'ind_ctop_fin_ult1_lag1',
 'ind_ctpp_fin_ult1_lag1',
 'ind_deco_fin_ult1_lag1',
 'ind_deme_fin_ult1_lag1',
 'ind_dela_fin_ult1_lag1',
 'ind_ecue_fin_ult1_lag1',
 'ind_fond_fin_ult1_lag1',
 'ind_hip_fin_ult1_lag1',
 'ind_plan_fin_ult1_lag1',
 'ind_pres_fin_ult1_lag1',
 'ind_reca_fin_ult1_lag1',
 'ind_tjcr_fin_ult1_lag1',
 

In [8]:
test_lag = {}
test_lag[5] = pd.read_csv(root + '/test_lag5_changename.csv')

In [12]:
test_lag[5].columns

Index([u'fecha_dato', u'ncodpers', u'pais_residencia', u'age', u'ind_nuevo',
       u'antiguedad', u'indrel', u'indrel_1mes', u'tiprel_1mes',
       u'canal_entrada',
       ...
       u'ind_recibo_ult1_lag5', u'age_lag5', u'ind_nuevo_lag5',
       u'antiguedad_lag5', u'indrel_1mes_lag5', u'tiprel_1mes_lag5',
       u'ind_actividad_cliente_lag5', u'card number_lag5',
       u'segmento_02 - PARTICULARES_lag5',
       u'segmento_03 - UNIVERSITARIO_lag5'],
      dtype='object', length=204)

In [27]:
product = list(new_each_month[5].filter(regex='^ind.*ult1$', axis=1))
product

['ind_ahor_fin_ult1',
 'ind_cco_fin_ult1',
 'ind_cder_fin_ult1',
 'ind_cno_fin_ult1',
 'ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1',
 'ind_ctop_fin_ult1',
 'ind_ctpp_fin_ult1',
 'ind_dela_fin_ult1',
 'ind_ecue_fin_ult1',
 'ind_fond_fin_ult1',
 'ind_hip_fin_ult1',
 'ind_nom_pens_ult1',
 'ind_nomina_ult1',
 'ind_plan_fin_ult1',
 'ind_pres_fin_ult1',
 'ind_reca_fin_ult1',
 'ind_recibo_ult1',
 'ind_tjcr_fin_ult1',
 'ind_valo_fin_ult1',
 'ind_viv_fin_ult1']

In [10]:
month = ['2015-01-28', '2015-02-28', '2015-03-28', '2015-04-28',
       '2015-05-28', '2015-06-28', '2015-07-28', '2015-08-28',
       '2015-09-28', '2015-10-28', '2015-11-28', '2015-12-28',
       '2016-01-28', '2016-02-28', '2016-03-28', '2016-04-28',
       '2016-05-28']
month

['2015-01-28',
 '2015-02-28',
 '2015-03-28',
 '2015-04-28',
 '2015-05-28',
 '2015-06-28',
 '2015-07-28',
 '2015-08-28',
 '2015-09-28',
 '2015-10-28',
 '2015-11-28',
 '2015-12-28',
 '2016-01-28',
 '2016-02-28',
 '2016-03-28',
 '2016-04-28',
 '2016-05-28']

In [11]:
reverse_month = month[::-1]
reverse_month

['2016-05-28',
 '2016-04-28',
 '2016-03-28',
 '2016-02-28',
 '2016-01-28',
 '2015-12-28',
 '2015-11-28',
 '2015-10-28',
 '2015-09-28',
 '2015-08-28',
 '2015-07-28',
 '2015-06-28',
 '2015-05-28',
 '2015-04-28',
 '2015-03-28',
 '2015-02-28',
 '2015-01-28']

In [12]:
month_name = ['Jan-2015', 'Feb-2015', 'Mar-2015', 'April-2015', 'May-2015', 'June-2015', 'July-2015', 'Aug-2015', 
             'Sep-2015', 'Oct-2015', 'Nov-2015', 'Dec-2015', 'Jan-2016', 'Feb-2016', 'Mar-2016', 'April-2016',
             'May-2016']

In [16]:
# ncol = new_each_month[5].filter(regex = '_lag').columns
# ncol
# sub =  new_each_month[5][new_each_month[5].columns[~new_each_month[5].columns.isin(ncol)]]

In [13]:
label = new_each_month[5].columns[new_each_month[5].columns.isin(product)]
new_each_month[5][label].sum(axis = 0)

ind_cco_fin_ult1     9454.0
ind_cder_fin_ult1       9.0
ind_cno_fin_ult1     1934.0
ind_ctju_fin_ult1      55.0
ind_ctma_fin_ult1     349.0
ind_ctop_fin_ult1     222.0
ind_ctpp_fin_ult1     154.0
ind_deco_fin_ult1     503.0
ind_dela_fin_ult1    1085.0
ind_deme_fin_ult1      33.0
ind_ecue_fin_ult1    1219.0
ind_fond_fin_ult1     246.0
ind_hip_fin_ult1        4.0
ind_nom_pens_ult1    8228.0
ind_nomina_ult1      5160.0
ind_plan_fin_ult1      21.0
ind_pres_fin_ult1       8.0
ind_reca_fin_ult1    2941.0
ind_recibo_ult1      9129.0
ind_tjcr_fin_ult1    4755.0
ind_valo_fin_ult1     159.0
ind_viv_fin_ult1        3.0
dtype: float64

In [14]:
June_15_lable = new_each_month[5][product]
June_15_lable['ncodpers'] = new_each_month[5]['ncodpers']
June_15_lable.set_index('ncodpers', inplace = True)
stacked_lables = June_15_lable.stack()
stacked_lables = stacked_lables.reset_index()
stacked_lables.columns = ['ncodpers', 'product', 'new']
stacked_lables

/Users/mengruiyin/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,ncodpers,product,new
0,15892,ind_cco_fin_ult1,1.0
1,15892,ind_cder_fin_ult1,0.0
2,15892,ind_cno_fin_ult1,0.0
3,15892,ind_ctju_fin_ult1,0.0
4,15892,ind_ctma_fin_ult1,0.0
5,15892,ind_ctop_fin_ult1,0.0
6,15892,ind_ctpp_fin_ult1,0.0
7,15892,ind_deco_fin_ult1,0.0
8,15892,ind_dela_fin_ult1,0.0
9,15892,ind_deme_fin_ult1,0.0


In [15]:
target_transform = stacked_lables[stacked_lables.new == 1]

In [16]:
target_transform.shape

(45671, 3)

In [26]:
col = sel_fea[:]
col.append('ncodpers')

In [42]:
len(col)

177

In [43]:
select_df = new_each_month[5][col]

In [45]:
#multi_class_June_15 = pd.merge(new_each_month[5], target_transform, on = 'ncodpers', how = 'inner')

multi_class_June_15 = pd.merge(select_df, target_transform, on = 'ncodpers', how = 'inner')

In [46]:
multi_class_June_15.sort_index().head(100)

,ind_deme_fin_ult1_lag5,ind_deme_fin_ult1_lag1,ind_viv_fin_ult1_lag3,ind_viv_fin_ult1_lag1,ind_viv_fin_ult1_lag5,ind_deco_fin_ult1_lag1,ind_aval_fin_ult1_lag2,ind_cder_fin_ult1_lag5,indresi_S,ind_ahor_fin_ult1_lag1,...,ind_recibo_ult1_lag5,age,nomprov,age_lag1,antiguedad_lag1,card number_lag5,renta,ncodpers,product,new
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,...,1.0,61,30,61.0,246.0,7.0,430477.41,15892,ind_cco_fin_ult1,1.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,...,NaN,49,30,NaN,NaN,NaN,183021.75,15896,ind_cco_fin_ult1,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,...,1.0,61,30,61.0,246.0,10.0,175450.05,15897,ind_nom_pens_ult1,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,...,1.0,55,30,55.0,227.0,7.0,81005.49,15906,ind_nom_pens_ult1,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,...,0.0,62,30,62.0,246.0,3.0,42831.69,15925,ind_recibo_ult1,1.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,...,1.0,62,30,62.0,245.0,5.0,278937.57,15927,ind_cco_fin_ult1,1.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,...,1.0,51,30,51.0,246.0,6.0,256366.71,15964,ind_reca_fin_ult1,1.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,...,1.0,57,30,57.0,246.0,5.0,493081.80,15982,ind_nom_pens_ult1,1.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,...,1.0,57,30,57.0,246.0,5.0,493081.80,15982,ind_nomina_ult1,1.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,...,1.0,53,30,53.0,245.0,4.0,151911.54,16026,ind_nom_pens_ult1,1.0


In [47]:
multi_class_June_15.set_index('ncodpers', inplace = True)

In [26]:
product_June_15 = new_each_month[5].columns[new_each_month[5].columns.isin(product)]

In [48]:
#multi_class_June_15 = multi_class_June_15.drop(columns = product_June_15)
multi_class_June_15.drop_duplicates(inplace = True)

In [49]:
#June_15_X = multi_class_June_15.drop(columns = ['product', 'new', 'fecha_dato', 'card number'])
June_15_X = multi_class_June_15.drop(columns = ['product', 'new'])
June_15_y = multi_class_June_15['product']

In [50]:
June_15_X.shape

(45668, 176)

In [51]:
June_15_y.value_counts()

ind_cco_fin_ult1     9451
ind_recibo_ult1      9129
ind_nom_pens_ult1    8228
ind_nomina_ult1      5160
ind_tjcr_fin_ult1    4755
ind_reca_fin_ult1    2941
ind_cno_fin_ult1     1934
ind_ecue_fin_ult1    1219
ind_dela_fin_ult1    1085
ind_deco_fin_ult1     503
ind_ctma_fin_ult1     349
ind_fond_fin_ult1     246
ind_ctop_fin_ult1     222
ind_valo_fin_ult1     159
ind_ctpp_fin_ult1     154
ind_ctju_fin_ult1      55
ind_deme_fin_ult1      33
ind_plan_fin_ult1      21
ind_cder_fin_ult1       9
ind_pres_fin_ult1       8
ind_hip_fin_ult1        4
ind_viv_fin_ult1        3
Name: product, dtype: int64

In [52]:
June_15_y = June_15_y.astype('category').cat.codes

In [53]:
train_June_15_X, test_June_15_X, train_June_15_y, test_June_15_y = train_test_split(June_15_X, June_15_y, test_size=0.3, random_state=42)

In [54]:
train_June_15_X.columns[train_June_15_X.dtypes == 'object']

Index([], dtype='object')

In [55]:
xgtrain = xgb.DMatrix(train_June_15_X, label = train_June_15_y)
xgtest = xgb.DMatrix(test_June_15_X , label = test_June_15_y)
watchlist = [(xgtrain, 'train'), (xgtest, 'eval')]

In [56]:
train_June_15_X.shape

(31967, 176)

In [41]:
# eval_metric = 'logloss',
#         num_class = 22,
#         nthread = 4,
#         eta = 0.04,
#         num_boost_round = 10,
#         max_depth = 6,
#         min_child_weight = 3,
#         subsample = 0.9,
#         colsample_bytree = 0.7,
#         silent = 1,
#         reg_alpha=0.07,
#         gamma = 0.8

In [57]:
random_state = 4

iterations = 40
printN = 1
#early_stopping_rounds = 10
params2 = {
        'eta': 0.04,
        'max_depth': 6,
        'min_child_weight': 3,
        'subsample': 0.9,
        'colsample_bytree': 0.7,
        'gamma': 0.8,
        'alpha': 0,
        'lambda': 1,
        'verbose_eval': True,
        'seed': random_state,
        'num_class': 22,
        'objective': "multi:softprob",
        'eval_metric': 'mlogloss',
        'num_boost_round': 50
    }

In [58]:
xgbModel2 = xgb.train(params2, 
                      xgtrain, 
                      iterations, 
                      watchlist,
                      verbose_eval = printN
                      #early_stopping_rounds=early_stopping_rounds
                      )

[0]	train-mlogloss:2.90259	eval-mlogloss:2.90586
[1]	train-mlogloss:2.75398	eval-mlogloss:2.7583
[2]	train-mlogloss:2.62987	eval-mlogloss:2.6353
[3]	train-mlogloss:2.52215	eval-mlogloss:2.52909
[4]	train-mlogloss:2.43166	eval-mlogloss:2.43998
[5]	train-mlogloss:2.35664	eval-mlogloss:2.36696
[6]	train-mlogloss:2.28195	eval-mlogloss:2.29334
[7]	train-mlogloss:2.21838	eval-mlogloss:2.23145
[8]	train-mlogloss:2.15943	eval-mlogloss:2.17394
[9]	train-mlogloss:2.10503	eval-mlogloss:2.12101
[10]	train-mlogloss:2.0512	eval-mlogloss:2.06851
[11]	train-mlogloss:2.00287	eval-mlogloss:2.02151
[12]	train-mlogloss:1.95675	eval-mlogloss:1.97677
[13]	train-mlogloss:1.91736	eval-mlogloss:1.93878
[14]	train-mlogloss:1.87902	eval-mlogloss:1.90157
[15]	train-mlogloss:1.84317	eval-mlogloss:1.86715
[16]	train-mlogloss:1.8091	eval-mlogloss:1.83426
[17]	train-mlogloss:1.778	eval-mlogloss:1.80435
[18]	train-mlogloss:1.74647	eval-mlogloss:1.77384
[19]	train-mlogloss:1.71571	eval-mlogloss:1.74431
[20]	train-mlogl

In [114]:
filename = 'xgbmodel_basic.sav'
pickle.dump(xgbModel2, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))

In [19]:
filename = 'xgbmodel_basic.sav'
xgbModel2 = pickle.load(open(filename, 'rb'))

In [18]:
filename2 = 'xgbmodel_improved.sav'
pickle.dump(xgbModel2, open(filename2, 'wb'))

NameError: name 'xgbModel2' is not defined

In [21]:
for kv in sorted([(k,v) for k,v in xgbModel2.get_fscore().items()], key=lambda kv: kv[1], reverse=True):
        print(kv)

('renta', 7029)
('nomprov', 2815)
('antiguedad_lag5', 2791)
('age_lag4', 2712)
('fecha_alta_month', 2462)
('fecha_alta_weekday', 2040)
('canal_entrada', 1778)
('card number_lag5', 1624)
('age', 1250)
('antiguedad_lag1', 1090)
('fecha_alta_year', 1057)
('card number_lag1', 1050)
('age_lag1', 994)
('card number_lag4', 970)
('card number_lag3', 858)
('card number_lag2', 837)
('age_lag5', 829)
('antiguedad_lag3', 797)
('age_lag3', 776)
('antiguedad', 734)
('sexo_V', 725)
('ind_cco_fin_ult1_lag5', 719)
('ind_recibo_ult1_lag5', 679)
('age_lag2', 674)
('antiguedad_years', 654)
('antiguedad_lag2', 647)
('ind_tjcr_fin_ult1_lag5', 560)
('bins', 506)
('ind_nom_pens_ult1_lag5', 482)
('ind_recibo_ult1_lag4', 453)
('antiguedad_lag4', 426)
('ind_cno_fin_ult1_lag5', 401)
('ind_recibo_ult1_lag3', 391)
('ind_recibo_ult1_lag2', 390)
('ind_nom_pens_ult1_lag4', 347)
('ind_recibo_ult1_lag1', 346)
('ind_nomina_ult1_lag5', 333)
('ind_tjcr_fin_ult1_lag1', 329)
('ind_tjcr_fin_ult1_lag4', 324)
('ind_tjcr_fin_ult

In [23]:
import operator
importance = xgbModel2.get_fscore()
importance = sorted(importance.items(), key=operator.itemgetter(1))
print (importance)
print (len(importance))

[('ind_aval_fin_ult1_lag5', 3), ('ind_aval_fin_ult1_lag1', 3), ('ind_pres_fin_ult1_lag2', 4), ('ind_aval_fin_ult1_lag3', 4), ('ind_hip_fin_ult1_lag3', 4), ('ind_deco_fin_ult1_lag1', 4), ('indresi_N', 5), ('ind_deme_fin_ult1_lag3', 5), ('ind_deme_fin_ult1_lag2', 6), ('ind_pres_fin_ult1_lag1', 8), ('ind_hip_fin_ult1_lag4', 8), ('ind_nuevo_lag5', 9), ('ind_aval_fin_ult1_lag2', 9), ('ind_ctju_fin_ult1_lag1', 9), ('ind_hip_fin_ult1_lag1', 9), ('ind_deme_fin_ult1_lag1', 10), ('ind_nuevo_lag4', 10), ('ind_deme_fin_ult1_lag5', 10), ('ind_hip_fin_ult1_lag5', 11), ('segmento_03 - UNIVERSITARIO_lag4', 12), ('segmento_03 - UNIVERSITARIO_lag3', 12), ('ind_viv_fin_ult1_lag1', 13), ('indext_N', 14), ('segmento_03 - UNIVERSITARIO_lag2', 17), ('ind_ctju_fin_ult1_lag2', 18), ('segmento_03 - UNIVERSITARIO', 19), ('ind_nuevo_lag3', 20), ('ind_plan_fin_ult1_lag1', 21), ('ind_hip_fin_ult1_lag2', 22), ('tiprel_1mes_lag1', 24), ('segmento_03 - UNIVERSITARIO_lag1', 25), ('ind_nuevo', 26), ('tiprel_1mes_lag2', 

In [24]:
sel_fea = []
for i in range(len(importance)):
    sel_fea.append(importance[i][0])

In [25]:
len(sel_fea)

166

In [37]:
test_lag = {}
test_lag[5] = pd.read_csv('/Volumes/My Passport for Mac/project2' + '/test_lag'+ str(5) + '_changename.csv', index_col = 0)

In [15]:
test_lag[5].head()

,ncodpers,pais_residencia,age,ind_nuevo,antiguedad,indrel,indrel_1mes,tiprel_1mes,canal_entrada,nomprov,...,ind_recibo_ult1_lag1,age_lag1,ind_nuevo_lag1,antiguedad_lag1,card number_lag1,indrel_1mes_lag1,tiprel_1mes_lag1,ind_actividad_cliente_lag1,segmento_02 - PARTICULARES_lag1,segmento_03 - UNIVERSITARIO_lag1
fecha_dato,,,,,,,,,,,,,,,,,,,,,
2016-06-28,15889,36,56,0,256.0,1,0,0,24,30,...,0.0,56.0,0.0,251.0,4.0,0.0,0.0,1.0,0.0,0.0
2016-06-28,1170544,36,36,0,34.0,1,0,1,24,2,...,0.0,35.0,0.0,29.0,1.0,0.0,1.0,0.0,1.0,0.0
2016-06-28,1170545,36,22,0,34.0,1,0,0,150,18,...,0.0,22.0,0.0,29.0,1.0,0.0,0.0,1.0,1.0,0.0
2016-06-28,1170547,36,22,0,34.0,1,0,1,150,8,...,0.0,22.0,0.0,29.0,1.0,0.0,1.0,0.0,0.0,0.0
2016-06-28,1170548,36,22,0,34.0,1,0,1,150,7,...,0.0,22.0,0.0,29.0,1.0,0.0,1.0,0.0,1.0,0.0


In [27]:
test = test_lag[5][col]

In [84]:
# test_lag[5].drop(columns = 'fecha_dato', inplace = True)

In [28]:
test.set_index('ncodpers', inplace = True)

In [29]:
test.fillna(0, inplace = True)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [31]:
test.head()

,ind_aval_fin_ult1_lag5,ind_aval_fin_ult1_lag1,ind_pres_fin_ult1_lag2,ind_aval_fin_ult1_lag3,ind_hip_fin_ult1_lag3,ind_deco_fin_ult1_lag1,indresi_N,ind_deme_fin_ult1_lag3,ind_deme_fin_ult1_lag2,ind_pres_fin_ult1_lag1,...,antiguedad_lag1,age,card number_lag5,canal_entrada,fecha_alta_weekday,fecha_alta_month,age_lag4,antiguedad_lag5,nomprov,renta
ncodpers,,,,,,,,,,,,,,,,,,,,,
15889,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,...,251.0,56,4.0,24,0,0,56.0,255.0,30,326124.90
1170544,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,...,29.0,36,1.0,24,2,7,35.0,33.0,2,67526.28
1170545,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,...,29.0,22,1.0,150,2,7,22.0,33.0,18,97689.30
1170547,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,...,29.0,22,1.0,150,2,7,22.0,33.0,8,148402.98
1170548,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,...,29.0,22,1.0,150,2,7,22.0,33.0,7,106885.80


In [62]:
test = test[train_June_15_X.columns]

In [30]:
test_lag[5].columns

Index(['ncodpers', 'pais_residencia', 'age', 'ind_nuevo', 'antiguedad',
       'indrel', 'indrel_1mes', 'tiprel_1mes', 'canal_entrada', 'nomprov',
       ...
       'ind_recibo_ult1_lag1', 'age_lag1', 'ind_nuevo_lag1', 'antiguedad_lag1',
       'card number_lag1', 'indrel_1mes_lag1', 'tiprel_1mes_lag1',
       'ind_actividad_cliente_lag1', 'segmento_02 - PARTICULARES_lag1',
       'segmento_03 - UNIVERSITARIO_lag1'],
      dtype='object', length=203)

In [63]:
train_June_15_X.columns.equals(test.columns)

True

In [64]:
xgtest_given = xgb.DMatrix(test)

In [65]:
prediction = xgbModel2.predict(xgtest_given)

In [113]:
np.save('test3.npy', prediction) 

In [114]:
product

['ind_cco_fin_ult1',
 'ind_cder_fin_ult1',
 'ind_cno_fin_ult1',
 'ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1',
 'ind_ctop_fin_ult1',
 'ind_ctpp_fin_ult1',
 'ind_deco_fin_ult1',
 'ind_dela_fin_ult1',
 'ind_deme_fin_ult1',
 'ind_ecue_fin_ult1',
 'ind_fond_fin_ult1',
 'ind_hip_fin_ult1',
 'ind_nom_pens_ult1',
 'ind_nomina_ult1',
 'ind_plan_fin_ult1',
 'ind_pres_fin_ult1',
 'ind_reca_fin_ult1',
 'ind_recibo_ult1',
 'ind_tjcr_fin_ult1',
 'ind_valo_fin_ult1',
 'ind_viv_fin_ult1']

In [66]:
sort_pred = np.fliplr(np.argsort(prediction, axis=1))

In [67]:
test_May16_pro = pd.read_csv(root + '/test_May16_product.csv')

In [68]:
May16_product = []
for id in range(test_May16_pro.shape[0]):
    a = test_May16_pro.iloc[id]
    b = list(a[a.values == 1].index)
    May16_product.append(list(b))
    if id % 100000 == 0:
        print id

0
100000
200000
300000
400000
500000
600000
700000
800000
900000


In [1]:
# popular_product = ['ind_cco_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_ecue_fin_ult1',
#                   'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1','ind_valo_fin_ult1', 'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

In [69]:
popular_product2 =['ind_cco_fin_ult1',
 'ind_cno_fin_ult1',
 'ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1',
 'ind_ctop_fin_ult1',
 'ind_ctpp_fin_ult1',
 'ind_ecue_fin_ult1',
 'ind_fond_fin_ult1',
 'ind_plan_fin_ult1',
 'ind_pres_fin_ult1',
 'ind_reca_fin_ult1',
 'ind_tjcr_fin_ult1',
 'ind_valo_fin_ult1',
 'ind_nomina_ult1',
 'ind_nom_pens_ult1',
 'ind_recibo_ult1']

In [70]:
new_product = []
for id in range(len(sort_pred)):
    temp = []
    for i in range(len(sort_pred[id])):
        if (product[sort_pred[id][i]] not in May16_product[id]) & (product[sort_pred[id][i]] in popular_product2):
            temp.append(product[sort_pred[id][i]])
        if len(temp) == 7:
            break
    new_product.append(' '.join(temp))

In [71]:
new_product[0]

'ind_recibo_ult1 ind_reca_fin_ult1 ind_nomina_ult1 ind_nom_pens_ult1 ind_ctop_fin_ult1 ind_ecue_fin_ult1 ind_fond_fin_ult1'

In [97]:
new_product[0]

'ind_recibo_ult1 ind_reca_fin_ult1 ind_nomina_ult1 ind_nom_pens_ult1 ind_ctop_fin_ult1 ind_ecue_fin_ult1 ind_fond_fin_ult1'

In [72]:
submission = pd.DataFrame({'ncodpers':test_lag[5].ncodpers,'added_products':new_product})

In [73]:
submission.to_csv(root + '/submission_name1212.csv', index = False)

In [110]:
submission.to_csv(root + '/submission_0359.csv', index = False)

In [47]:
submission = pd.read_csv(root + '/submission_0359.csv')

In [49]:
submission['added_products'].iloc[0]

'ind_recibo_ult1 ind_reca_fin_ult1 ind_nomina_ult1 ind_nom_pens_ult1 ind_ctop_fin_ult1 ind_ecue_fin_ult1 ind_fond_fin_ult1'

In [117]:
fitted = xgbModel2.predict(xgtest)

In [119]:
sort_fitted = np.fliplr(np.argsort(fitted, axis=1))

In [148]:
product[2]

'ind_cno_fin_ult1'

In [144]:
product[17]

'ind_reca_fin_ult1'

In [131]:
product[13]

'ind_nom_pens_ult1'

In [147]:
sort_fitted[12]

array([18, 13, 14,  2, 17, 10, 19,  8, 20, 11,  0,  4, 15,  9,  1, 16, 12,
        5,  7,  6, 21,  3])

In [134]:
pd.DataFrame(test_June_15_y).reset_index().head(50)

,ncodpers,0
0,44348,18
1,108124,0
2,199080,18
3,1082351,0
4,1222073,2
5,1400038,0
6,1284200,13
7,1038513,2
8,1401211,0
9,107976,17


##### tune parameter

In [35]:
class XGBoostClassifier():
    def __init__(self, num_boost_round=10, **params):
        self.clf = None
        self.num_boost_round = num_boost_round
        self.params = params
        self.params.update({'objective': 'multi:softprob'})
 
    def fit(self, X, y, num_boost_round=None):
        num_boost_round = num_boost_round or self.num_boost_round
        self.label2num = dict((label, i) for i, label in enumerate(sorted(set(y))))
        dtrain = xgb.DMatrix(X, label=[self.label2num[label] for label in y])
        self.clf = xgb.train(params=self.params, dtrain=dtrain, num_boost_round=num_boost_round)
 
    def predict(self, X):
        num2label = dict((i, label)for label, i in self.label2num.items())
        Y = self.predict_proba(X)
        y = np.argmax(Y, axis=1)
        return np.array([num2label[i] for i in y])
 
    def predict_proba(self, X):
        dtest = xgb.DMatrix(X)
        return self.clf.predict(dtest)
 
    def score(self, X, y):
        Y = self.predict_proba(X)
        return 1 / logloss(y, Y)
 
    def get_params(self, deep=True):
        return self.params
 
    def set_params(self, **params):
        if 'num_boost_round' in params:
            self.num_boost_round = params.pop('num_boost_round')
        if 'objective' in params:
            del params['objective']
        self.params.update(params)
        return self
    
    
def logloss(y_true, Y_pred):
    label2num = dict((name, i) for i, name in enumerate(sorted(set(y_true))))
    return -1 * sum(math.log(y[label2num[label]]) if y[label2num[label]] > 0 else -np.inf for y, label in zip(Y_pred, y_true)) / len(Y_pred)


In [36]:
clf = XGBClassifier(
        eval_metric = 'logloss',
        num_class = 22,
        nthread = 4,
        eta = 0.04,
        num_boost_round = 10,
        max_depth = 6,
        min_child_weight = 3,
        subsample = 0.9,
        colsample_bytree = 0.7,
        silent = 1,
        reg_alpha=0.07,
        gamma = 0.8
        )

parameters = {
        #'gamma':[0.6, 0.7,0.8],
        #'num_boost_round': [50, 100, 150],
        #'eta': [0.04,0.05,0.06, 0.07]
        #'max_depth': [6,7,8],
        #'min_child_weight':[1,2,3],
        #'subsample': [0.7, 0.8, 0.9]
        #'colsample_bytree': [0.7,0.8],
        'num_boost_round': [50, 100, 150]
     }

clf = GridSearchCV(clf, parameters, n_jobs=1, cv=2)
print("Fitting the model")
clf.fit(train_June_15_X,train_June_15_y)
print("Training model completed")
best_parameters, score, _ = max(clf.grid_scores_, key=lambda x: x[1])
print(score)

Fitting the model


/Users/mengruiyin/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/mengruiyin/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/mengruiyin/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/mengruiyin/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an 

Training model completed
0.6278349547971346


/Users/mengruiyin/anaconda/lib/python2.7/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [160]:
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

max_depth: 6


In [37]:
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

min_child_weight: 3


In [40]:
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

gamma: 0.8


In [43]:
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

colsample_bytree: 0.7


In [45]:
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

subsample: 0.9


In [47]:
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

eta: 0.04


In [37]:
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

num_boost_round: 50


In [92]:
a = ['ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1','ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1','ind_dela_fin_ult1','ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1','ind_nom_pens_ult1','ind_recibo_ult1']

['ind_cco_fin_ult1',
 'ind_cder_fin_ult1',
 'ind_cno_fin_ult1',
 'ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1',
 'ind_ctop_fin_ult1',
 'ind_ctpp_fin_ult1',
 'ind_deco_fin_ult1',
 'ind_deme_fin_ult1',
 'ind_dela_fin_ult1',
 'ind_ecue_fin_ult1',
 'ind_fond_fin_ult1',
 'ind_hip_fin_ult1',
 'ind_plan_fin_ult1',
 'ind_pres_fin_ult1',
 'ind_reca_fin_ult1',
 'ind_tjcr_fin_ult1',
 'ind_valo_fin_ult1',
 'ind_viv_fin_ult1',
 'ind_nomina_ult1',
 'ind_nom_pens_ult1',
 'ind_recibo_ult1']